In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import chart_studio
import chart_studio.plotly as py
import plotly.express as px
import kaleido

In [2]:
state_filepath='/Users/melodyqian/Documents/GitHub/FindMyNuclearWaste/CSVs/State_Level_Demographics_filtered.csv'
statedf=pd.read_csv(state_filepath)

In [3]:
pd.DataFrame.head(statedf, n=10)

,state_name,state_abbrev,house_value_median,household_income_median,age_median,male_percent,female_percent,latino_percent,white_percent,black_percent,native_percent,asian_percent,pacific_percent
0,Alabama,AL,195100,62027,39.3,48.5,51.5,27.5,65.4,26.1,0.4,1.6,0.3
1,Arizona,AZ,358900,76872,38.8,49.9,50.1,6.4,63.2,4.6,4.1,3.3,0.2
2,Arkansas,AR,175300,58773,38.4,49.3,50.7,16.4,70.9,14.9,0.6,2.2,0.1
3,California,CA,695400,96334,37.6,50.0,50.0,7.2,44.0,5.5,1.1,6.7,1.2
4,Colorado,CO,502200,92470,37.5,50.6,49.4,5.7,73.5,4.0,1.0,5.5,0.4
5,Connecticut,CT,343200,93760,41.2,49.1,50.9,13.6,67.6,10.7,0.3,7.4,0.3
6,Florida,FL,325000,71711,42.6,49.1,50.9,17.8,59.9,15.3,0.3,10.8,0.1
7,Georgia,GA,272900,74664,37.4,48.8,51.2,33.7,52.5,31.3,0.4,3.7,0.6
8,Idaho,ID,376000,74636,37.1,50.3,49.7,1.4,83.3,0.7,1.1,2.1,0.1
9,Illinois,IL,250500,81702,38.9,49.4,50.6,15.5,63.3,13.8,0.5,2.4,0.4


In [4]:
fig = px.bar(statedf, x="state_name", y=["asian_percent"], title="Wide-Form Input")
fig.show()
#fig.write_html("samplestatedata.html")

In [ ]:
state_income_collapsed = statedf.groupby('state_abbrev')['household_income_median'].mean()
state_income_collapsed= state_income_collapsed.reset_index()

In [6]:
pd.DataFrame.head(state_income_collapsed, n=10)

,state_abbrev,household_income_median
0,AL,62027.0
1,AR,58773.0
2,AZ,76872.0
3,CA,96334.0
4,CO,92470.0
5,CT,93760.0
6,FL,71711.0
7,GA,74664.0
8,IA,73147.0
9,ID,74636.0


In [7]:
site_filepath='/Users/melodyqian/Documents/GitHub/FindMyNuclearWaste/CSVs/DemographicEconomicInfo.csv'
sitedf=pd.read_csv(site_filepath)

In [8]:
site_income_collapsed = sitedf.groupby('state_abbrev')['household_income_median'].mean()
site_income_collapsed= site_income_collapsed.reset_index()

In [9]:
import plotly.graph_objects as go

# Merge the two dataframes on state_abbrev
comparison_df = pd.merge(
    state_income_collapsed[['state_abbrev', 'household_income_median']], 
    site_income_collapsed[['state_abbrev', 'household_income_median']], 
    on='state_abbrev', 
    suffixes=('_state', '_site')
)

# Sort by one of the values (optional)
comparison_df = comparison_df.sort_values('household_income_median_state')



In [10]:
pd.DataFrame.head(comparison_df, n=10)


,state_abbrev,household_income_median_state,household_income_median_site
19,MS,54915.0,15104.000000
1,AR,58773.0,67063.000000
12,LA,60023.0,159357.500000
0,AL,62027.0,71468.000000
24,NM,62125.0,27564.000000
29,SC,66818.0,82501.000000
30,TN,67097.0,67656.000000
18,MO,68920.0,61382.000000
26,OH,69680.0,85376.500000
20,NC,69904.0,132112.333333


In [11]:
comparison_df = comparison_df.sort_values('household_income_median_state')
# Create the lollipop plot
site_fig = go.Figure()

# Add lines connecting the two points
for i in range(len(comparison_df)):
    site_fig.add_trace(go.Scatter(
        x=[comparison_df.iloc[i]['household_income_median_state'], 
           comparison_df.iloc[i]['household_income_median_site']],
        y=[comparison_df.iloc[i]['state_abbrev'], comparison_df.iloc[i]['state_abbrev']],
        mode='lines',
        line=dict(color='lightgray', width=2),
        showlegend=False,
        hoverinfo='skip'
    ))

# Add points for statedf
site_fig.add_trace(go.Scatter(
    x=comparison_df['household_income_median_state'],
    y=comparison_df['state_abbrev'],
    mode='markers',
    name='State Level',
    marker=dict(size=10, color='blue'),
    hovertemplate='<b>%{y}</b><br>State: $%{x:,.0f}<extra></extra>'
))

# Add points for sitedf
site_fig.add_trace(go.Scatter(
    x=comparison_df['household_income_median_site'],
    y=comparison_df['state_abbrev'],
    mode='markers',
    name='Site Level',
    marker=dict(size=10, color='red'),
    hovertemplate='<b>%{y}</b><br>Site: $%{x:,.0f}<extra></extra>'
))

# Update layout
site_fig.update_layout(
    title='Comparison of Median Household Income by State',
    xaxis_title='Median Household Income ($)',
    yaxis_title='State',
    height=600,
    hovermode='closest',
    showlegend=True
)

site_fig.show()

In [12]:
site_fig.write_image("income_lollipop.png")

In [19]:
state_house_collapsed = statedf.groupby('state_abbrev')['house_value_median'].mean()
state_house_collapsed= state_house_collapsed.reset_index()
site_house_collapsed = sitedf.groupby('state_abbrev')['house_value_median'].mean()
site_house_collapsed= site_house_collapsed.reset_index()

import plotly.graph_objects as go

# Merge the two dataframes on state_abbrev
comparehouse_df = pd.merge(
    state_house_collapsed[['state_abbrev', 'house_value_median']], 
    site_house_collapsed[['state_abbrev', 'house_value_median']], 
    on='state_abbrev', 
    suffixes=('_state', '_site')
)

# Sort by one of the values (optional)
comparehouse_df = comparehouse_df.sort_values('house_value_median_state')

In [20]:
housesite_fig = go.Figure()

# Add lines connecting the two points
for i in range(len(comparehouse_df)):
    housesite_fig.add_trace(go.Scatter(
        x=[comparehouse_df.iloc[i]['house_value_median_state'], 
           comparehouse_df.iloc[i]['house_value_median_site']],
        y=[comparehouse_df.iloc[i]['state_abbrev'], comparehouse_df.iloc[i]['state_abbrev']],
        mode='lines',
        line=dict(color='lightgray', width=2),
        showlegend=False,
        hoverinfo='skip'
    ))

# Add points for statedf
housesite_fig.add_trace(go.Scatter(
    x=comparehouse_df['house_value_median_state'],
    y=comparehouse_df['state_abbrev'],
    mode='markers',
    name='State Level',
    marker=dict(size=10, color='blue'),
    hovertemplate='<b>%{y}</b><br>State: $%{x:,.0f}<extra></extra>'
))

# Add points for sitedf
housesite_fig.add_trace(go.Scatter(
    x=comparehouse_df['house_value_median_site'],
    y=comparehouse_df['state_abbrev'],
    mode='markers',
    name='Site Level',
    marker=dict(size=10, color='red'),
    hovertemplate='<b>%{y}</b><br>Site: $%{x:,.0f}<extra></extra>'
))

# Update layout
housesite_fig.update_layout(
    title='Comparison of Median Household Value by State',
    xaxis_title='Median Household Value ($)',
    yaxis_title='State',
    height=600,
    hovermode='closest',
    showlegend=True
)

housesite_fig.show()

In [21]:
housesite_fig.write_image("housevalue_lollipop.png")
